In [1]:
import pandas
from openeye import oechem, oespruce

# Loading protein, splitting, and aligning to reference

In [2]:
pdb_fn = "input_pdbs/5WKK.pdb"
## Load molecule to extract
ifs = oechem.oemolistream()
ifs.SetFlavor(
    oechem.OEFormat_PDB,
    oechem.OEIFlavor_PDB_Default
    | oechem.OEIFlavor_PDB_DATA
    | oechem.OEIFlavor_PDB_ALTLOC,
)
ifs.open(pdb_fn)
complex_mol = oechem.OEGraphMol()
oechem.OEReadMolecule(ifs, complex_mol)
ifs.close()

## Keep only highest occupancy alt location
alf = oechem.OEAltLocationFactory(complex_mol)
if alf.GetGroupCount() != 0:
    print(complex_mol.NumAtoms())
    alf.MakePrimaryAltMol(complex_mol)
    print(complex_mol.NumAtoms())

2634
2603


In [3]:
## Test splitting
lig_mol = oechem.OEGraphMol()
prot_mol = oechem.OEGraphMol()
water_mol = oechem.OEGraphMol()
oth_mol = oechem.OEGraphMol()

## Make splitting split out covalent ligands
## TODO: look into different covalent-related options here
opts = oechem.OESplitMolComplexOptions()
opts.SetSplitCovalent(True)
opts.SetSplitCovalentCofactors(True)
print(
    oechem.OESplitMolComplex(lig_mol, prot_mol, water_mol, oth_mol, complex_mol)
)

print(
    complex_mol.NumAtoms(),
    lig_mol.NumAtoms(),
    prot_mol.NumAtoms(),
    water_mol.NumAtoms(),
    oth_mol.NumAtoms(),
)

True
2603 62 2256 24 261


In [4]:
## Load ref PDB molecule
ref_fn = "input_pdbs/reference.pdb"
ref_complex_mol = oechem.OEGraphMol()
ifs.open(ref_fn)
oechem.OEReadMolecule(ifs, ref_complex_mol)
ifs.close()
print(ref_complex_mol.NumAtoms())

## Perform splitting for ref
ref_lig_mol = oechem.OEGraphMol()
ref_prot_mol = oechem.OEGraphMol()
ref_water_mol = oechem.OEGraphMol()
ref_oth_mol = oechem.OEGraphMol()
print(
    oechem.OESplitMolComplex(
        ref_lig_mol, ref_prot_mol, ref_water_mol, ref_oth_mol, ref_complex_mol
    )
)
print(
    ref_complex_mol.NumAtoms(),
    ref_lig_mol.NumAtoms(),
    ref_prot_mol.NumAtoms(),
    ref_water_mol.NumAtoms(),
    ref_oth_mol.NumAtoms(),
)

2832
True
2832 34 2316 18 464


In [5]:
## Align new protein to ref
aln_res = oespruce.OESuperposeResults()
superpos = oespruce.OESuperpose()
superpos.SetupRef(ref_prot_mol)
superpos.Superpose(aln_res, prot_mol)
print(f"RMSD: {aln_res.GetRMSD()}")

prot_mol_aligned = prot_mol.CreateCopy()
print(aln_res.Transform(prot_mol_aligned))

## Save new aligned molecule for inspection
out_fn = "test/5WKK_split_aligned.pdb"
ofs = oechem.oemolostream()
ofs.SetFlavor(oechem.OEFormat_PDB, oechem.OEOFlavor_PDB_Default)
ofs.open(out_fn)
oechem.OEWriteMolecule(ofs, prot_mol_aligned)
ofs.close()

RMSD: 1.8288194400350355
True


# Load in new ligand and create DesignUnit

In [6]:
## Load ligand molecule
sdf_fn = f"fragalysis/aligned/Mpro-P2007_0A/Mpro-P2007_0A.sdf"
ifs = oechem.oemolistream()
ifs.SetFlavor(
    oechem.OEFormat_SDF,
    oechem.OEIFlavor_SDF_Default,
)
ifs.open(sdf_fn)
coords_mol = oechem.OEGraphMol()
oechem.OEReadMolecule(ifs, coords_mol)
ifs.close()
print(coords_mol.NumAtoms())

## Load CDD compound tracker
compound_df = pandas.read_csv(
    "fragalysis/extra_files/Mpro_compound_tracker_csv.csv"
)

## Get SMILES from compound tracker
idx = compound_df["Dataset"] == "Mpro-P2007"
smiles = compound_df.loc[idx, "SMILES"].values[0]
new_lig_mol = oechem.OEGraphMol()
oechem.OESmilesToMol(new_lig_mol, smiles)
print(new_lig_mol.NumAtoms())

## Reorder the atoms and bonds so the iterate together
oechem.OECanonicalOrderAtoms(coords_mol)
oechem.OECanonicalOrderBonds(coords_mol)
oechem.OECanonicalOrderAtoms(new_lig_mol)
oechem.OECanonicalOrderBonds(new_lig_mol)

20
20


In [7]:
## Copy coordinates from the SDF-loaded mol into the SMILES-generated one
## Build new coordinate dict with the proper numbering for the
##  smiles-based molecule
new_lig_mol_coords = {}
coords_mol_coords = coords_mol.GetCoords()
for a1, a2 in zip(new_lig_mol.GetAtoms(), coords_mol.GetAtoms()):
    idx1 = a1.GetIdx()
    idx2 = a2.GetIdx()
    new_lig_mol_coords[idx2] = coords_mol_coords[idx1]
new_lig_mol.SetCoords(new_lig_mol_coords)

True

# Brief aside to see if transferring coordinates works
(run to here)

In [ ]:
from openeye import oedepict

oedepict.OEPrepareDepiction(new_lig_mol_test)
oedepict.OEPrepareDepiction(smiles_mol_test)
width, height = 600, 600

opts = oedepict.OE2DMolDisplayOptions(width, height, oedepict.OEScale_AutoScale)
opts.SetAtomPropertyFunctor(oedepict.OEDisplayAtomIdx())
opts.SetAtomPropLabelFont(oedepict.OEFont(oechem.OEDarkGreen))

disp = oedepict.OE2DMolDisplay(new_lig_mol_test, opts)
oedepict.OERenderMolecule("test1.png", disp)

disp = oedepict.OE2DMolDisplay(smiles_mol_test, opts)
oedepict.OERenderMolecule("test2.png", disp)

In [ ]:
## Build new coordinate dict with the proper numbering for the
##  smiles-based molecule
smiles_mol_coords = {}
new_lig_mol_coords = new_lig_mol_test.GetCoords()
for a1, a2 in zip(new_lig_mol_test.GetAtoms(), smiles_mol_test.GetAtoms()):
    idx1 = a1.GetIdx()
    idx2 = a2.GetIdx()
    smiles_mol_coords[idx2] = new_lig_mol_coords[idx1]
smiles_mol_test.SetCoords(smiles_mol_coords)

## Add Hs to both to be able to see double bonds in PDB
print(oechem.OEAddExplicitHydrogens(new_lig_mol_test))
print(oechem.OEAddExplicitHydrogens(smiles_mol_test))
## Update H coordinates to be 3D (not sure why this is necessary,
##  I think only for smiles_mol_test)
print(oechem.OESet3DHydrogenGeom(new_lig_mol_test))
print(oechem.OESet3DHydrogenGeom(smiles_mol_test))

## Save both as PDB to load and see if it worked
ofs = oechem.oemolostream()
ofs.SetFlavor(oechem.OEFormat_PDB, oechem.OEOFlavor_PDB_Default)

ofs.open("test/original_ligand.pdb")
oechem.OEWriteMolecule(ofs, new_lig_mol_test)
ofs.close()

ofs.open("test/coord_copied_ligand.pdb")
oechem.OEWriteMolecule(ofs, smiles_mol_test)
ofs.close()
## It works!

# End aside
(run from here)

In [8]:
## Add explicit Hs to protein and new ligand to prep
prot_mol_prep = prot_mol_aligned.CreateCopy()
new_lig_prep = new_lig_mol.CreateCopy()
print(oechem.OEAddExplicitHydrogens(prot_mol_prep))
print(oechem.OEAddExplicitHydrogens(new_lig_prep))
## Update H coordinates to be 3D (not sure why this is necessary)
print(oechem.OESet3DHydrogenGeom(new_lig_prep))

## Re-reorder the atoms and bonds
oechem.OECanonicalOrderAtoms(new_lig_prep)
oechem.OECanonicalOrderBonds(new_lig_prep)

True
True
True


In [9]:
## Make the design unit and save
du = oechem.OEDesignUnit()
print(oespruce.OEMakeDesignUnit(du, prot_mol_prep, new_lig_prep))
print(du.HasProtein(), du.HasLigand())
du_out_fn = "test/5WKK_new_lig.oedu"
if du.HasProtein() and du.HasLigand():
    oechem.OEWriteDesignUnit(du_out_fn, du)

True
True True


   Falling back to charging ligand with OEMMFF94Charges


In [10]:
## Save protein + new ligand as PDB
new_complex_mol = oechem.OEGraphMol()
print(du.GetProtein(new_complex_mol))
print(new_complex_mol.NumAtoms())
print(du.GetLigand(new_complex_mol))
print(new_complex_mol.NumAtoms())
print(
    du.GetComponents(
        new_complex_mol,
        oechem.OEDesignUnitComponents_Protein
        | oechem.OEDesignUnitComponents_Ligand,
    )
)
print(new_complex_mol.NumAtoms())
ofs.open("test/5WKK_new_lig.pdb")
oechem.OEWriteMolecule(ofs, new_complex_mol)
ofs.close()

True
4582
True
32
True
4614


# Re-docking of new ligand

In [11]:
from kinoml.docking.OEDocking import pose_molecules

# score_pose = True
docked_molecules = pose_molecules(du, [new_lig_prep])
print(docked_molecules)

/home/kaminowb/miniconda3/envs/structure_ml/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/kaminowb/miniconda3/envs/structure_ml/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/kaminowb/miniconda3/envs/structure_ml/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/kaminowb/miniconda3/envs/structure_ml/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


None


In [12]:
mol = new_lig_prep.CreateCopy()
# print(oechem.OEClearChiralPerception(mol))
# print(oechem.OEPerceiveChiral(mol, True))
print(oechem.OE3DToAtomStereo(mol))

False


In [13]:
print(mol.HasPerceived(oechem.OEPerceived_Chiral))
print(oechem.OEClearChiralPerception(mol))
print(mol.HasPerceived(oechem.OEPerceived_Chiral))
print(oechem.OEPerceiveChiral(mol))
print(mol.HasPerceived(oechem.OEPerceived_Chiral))
for a in mol.GetAtoms():
    if a.IsChiral():
        print(a)

False
None
False
True
True


In [14]:
test_mol = new_lig_mol.CreateCopy()
oechem.OEAssignAromaticFlags(test_mol)
for a in test_mol.GetAtoms():
    print(a, a.GetImplicitHCount(), a.IsAromatic())

 0 C 1 True
 1 C 1 True
 2 C 1 True
 3 C 1 True
 4 C 1 True
 5 C 1 True
 6 C 1 True
 7 C 1 True
 8 C 1 True
 9 C 2 False
10 C 0 True
11 C 0 True
12 C 0 True
13 C 0 True
14 C 0 False
15 N 0 True
16 N 1 False
17 N 0 True
18 O 0 False
19Cl 0 False
